# The above code was for help, now I know what's going on. 

### Let's do this

In [1]:
# ============================================================
# 0. Imports & global settings
# ============================================================

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

MOD = 97  # modulus

# For reproducibility
torch.manual_seed(0)
np.random.seed(0)

Using device: cpu


In [2]:
import matplotlib as mpl

mpl.rcParams.update({
    "figure.dpi": 300,               # display DPI (useful for interactive), but saving uses savefig's dpi
    "savefig.dpi": 300,              # default saved raster DPI
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],
    "mathtext.fontset": "cm",        # or "stix" for Times-like math
    "axes.titlesize": 11,
    "axes.labelsize": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 8,
    "lines.linewidth": 1.0,
    "lines.markersize": 5,
    "axes.linewidth": 0.8,
    "axes.grid": False,
})


In [3]:
# ============================================================
# 1. Dataset: modular addition / multiplication on Z_97
# ============================================================

class ModArithDataset(Dataset):
    """
    Generates all pairs (a, b) in {0..MOD-1}^2
    with label f(a,b) = a [+ or *] b (mod MOD).
    """
    def __init__(self, op="add"):
        assert op in ["add", "mul"]
        self.op = op

        xs = []
        ys = []
        for a in range(MOD):
            for b in range(MOD):
                xs.append((a, b))
                if op == "add":
                    ys.append((a + b) % MOD)
                else:
                    ys.append((a * b) % MOD)

        self.x = torch.tensor(xs, dtype=torch.long)  # [N, 2]
        self.y = torch.tensor(ys, dtype=torch.long)  # [N]

    def __len__(self):
        return self.x.size(0)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

print("Dataset sizes:")
print("Addition:", len(ModArithDataset("add")))
print("Multiplication:", len(ModArithDataset("mul")))

Dataset sizes:
Addition: 9409
Multiplication: 9409


In [4]:
# ============================================================
# 2. Models: BilinearMLP and ReluMLP
# ============================================================

class BilinearMLP(nn.Module):
    """
    Two-input bilinear MLP:
      e_a = Emb_a[a], e_b = Emb_b[b]
      u = W1(e_a), v = W2(e_b)
      h = u * v (Hadamard)
      logits = W_out(h)
    """
    def __init__(self, num_tokens=MOD, d=32, m=64):
        super().__init__()
        self.emb_a = nn.Embedding(num_tokens, d)
        self.emb_b = nn.Embedding(num_tokens, d)
        self.W1 = nn.Linear(d, m, bias=False)
        self.W2 = nn.Linear(d, m, bias=False)
        self.out = nn.Linear(m, num_tokens, bias=False)

    def forward(self, a, b):
        # a, b: [B] long
        e_a = self.emb_a(a)   # [B, d]
        e_b = self.emb_b(b)   # [B, d]
        u = self.W1(e_a)      # [B, m]
        v = self.W2(e_b)      # [B, m]
        h = u * v             # bilinear interaction
        logits = self.out(h)  # [B, MOD]
        return logits


class ReluMLP(nn.Module):
    """
    Baseline ReLU MLP:
      e_a, e_b embeddings
      x = [e_a; e_b]
      h = ReLU(W(x))
      logits = W_out(h)
    """
    def __init__(self, num_tokens=MOD, d=32, m=64):
        super().__init__()
        self.emb_a = nn.Embedding(num_tokens, d)
        self.emb_b = nn.Embedding(num_tokens, d)
        self.fc1 = nn.Linear(2 * d, m)
        self.out = nn.Linear(m, num_tokens)

    def forward(self, a, b):
        e_a = self.emb_a(a)
        e_b = self.emb_b(b)
        x = torch.cat([e_a, e_b], dim=-1)  # [B, 2d]
        h = F.relu(self.fc1(x))            # [B, m]
        logits = self.out(h)               # [B, MOD]
        return logits

In [5]:
# ============================================================
# 3. Generic training loop
# ============================================================

def train_model(model, dataset, epochs=200, batch_size=256, lr=1e-3, verbose_every=20):
    model = model.to(device)
    N = len(dataset)
    n_train = int(0.8 * N)
    n_val = N - n_train
    train_ds, val_ds = random_split(dataset, [n_train, n_val])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    history = {"train_loss": [], "val_acc": []}

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for (x, y) in train_loader:
            a, b = x[:, 0].to(device), x[:, 1].to(device)
            y = y.to(device)
            logits = model(a, b)
            loss = criterion(logits, y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * y.size(0)
        train_loss = total_loss / n_train

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for (x, y) in val_loader:
                a, b = x[:, 0].to(device), x[:, 1].to(device)
                y = y.to(device)
                logits = model(a, b)
                pred = logits.argmax(dim=1)
                correct += (pred == y).sum().item()
                total += y.size(0)
        val_acc = correct / total

        history["train_loss"].append(train_loss)
        history["val_acc"].append(val_acc)

        if (epoch % verbose_every) == 0 or epoch == epochs - 1:
            print(f"[epoch {epoch:3d}] loss={train_loss:.4f}  val_acc={val_acc:.4f}")
            if(val_acc>0.999):
                print("Early Stopping Triggered!")
                break

    return model, history

In [ ]:
# ============================================================
# 4. Train models for ADDITION
# ============================================================

add_dataset = ModArithDataset(op="add")

bilinear_add = BilinearMLP(d=32, m=64)
relu_add     = ReluMLP(d=32, m=64)

print("Training Bilinear MLP on addition...")
bilinear_add, hist_b_add = train_model(bilinear_add, add_dataset, epochs=2000)

print("\nTraining ReLU MLP on addition...")
relu_add, hist_r_add = train_model(relu_add, add_dataset, epochs=2000)

In [ ]:
# ============================================================
# 5. Train models for MULTIPLICATION
# ============================================================

mul_dataset = ModArithDataset(op="mul")

bilinear_mul = BilinearMLP(d=32, m=64)
relu_mul     = ReluMLP(d=32, m=64)

print("Training Bilinear MLP on multiplication...")
bilinear_mul, hist_b_mul = train_model(bilinear_mul, mul_dataset, epochs=2000)

print("\nTraining ReLU MLP on multiplication...")
relu_mul, hist_r_mul = train_model(relu_mul, mul_dataset, epochs=2000)

In [6]:
# torch.save(bilinear_add, "/kaggle/working/bilinear_add.pth")
# torch.save(bilinear_mul, "/kaggle/working/bilinear_mul.pth")
# torch.save(relu_mul, "/kaggle/working/relu_mul.pth")
# torch.save(relu_add, "/kaggle/working/relu_add.pth")

bilinear_add = torch.load('bilinear_add.pth')
bilinear_mul = torch.load('bilinear_mul.pth')
relu_add = torch.load('relu_add.pth')
relu_mul = torch.load('relu_mul.pth')

C:\Users\Aditya\AppData\Local\Temp\ipykernel_25696\2477891122.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bilinear_add = torch.load('bilinear_add.pth')


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# ============================================================
# 6. Utility: build full logits grid for all (a,b)
# ============================================================

def full_logits_grid(model):
    """
    Return logits[a,b,k] for all a,b ∈ {0..MOD-1}.
    model is already trained.
    """
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        a_idx = torch.arange(MOD, device=device)
        b_idx = torch.arange(MOD, device=device)
        A, B = torch.meshgrid(a_idx, b_idx, indexing="ij")  # [MOD,MOD]
        A_flat = A.reshape(-1)
        B_flat = B.reshape(-1)
        logits = model(A_flat, B_flat)  # [MOD*MOD, MOD]
        logits = logits.reshape(MOD, MOD, MOD)  # [a,b,k]
    return logits.cpu().numpy()


In [ ]:
# ============================================================
# 7. Extract Mk from Bilinear weights (weight-level)
# ============================================================

def extract_Mk_bilinear_from_weights(model: BilinearMLP):
    """
    Analytic extraction of M_k[a,b] from bilinear weights.
    Returns Mk: numpy array of shape [MOD, MOD, MOD] where Mk[a,b,k].
    """
    model = model.to(device)
    with torch.no_grad():
        # Embeddings: [MOD, d]
        E_a = model.emb_a.weight.data.clone().to(device)
        E_b = model.emb_b.weight.data.clone().to(device)

        # Linear weights: nn.Linear stores weight [out_dim, in_dim]
        # We want W1: [d,m], W2: [d,m], out: [m,MOD]
        W1 = model.W1.weight.data.clone().to(device).T   # [d, m]
        W2 = model.W2.weight.data.clone().to(device).T   # [d, m]
        O  = model.out.weight.data.clone().to(device).T  # [m, MOD]

        # U[a,j] = e_a W1[:,j]
        U = E_a @ W1      # [MOD, m]
        V = E_b @ W2      # [MOD, m]

        U_np = U.cpu().numpy()   # [MOD, m]
        V_np = V.cpu().numpy()   # [MOD, m]
        O_np = O.cpu().numpy()   # [m, MOD]

    # hidden_pair[a,b,j] = U[a,j] * V[b,j]
    A = U_np[:, None, :]    # [MOD, 1, m]
    B = V_np[None, :, :]    # [1, MOD, m]
    hidden_pair = A * B     # [MOD, MOD, m]

    # Contract hidden dimension with out weights:
    # Mk[a,b,k] = sum_j hidden_pair[a,b,j] * O[j,k]
    Mk = np.einsum("abm,mk->abk", hidden_pair, O_np)  # [MOD, MOD, MOD]
    return Mk


In [ ]:
# ============================================================
# 8. Alternate: Mk directly from logits for ANY model
# ============================================================

def extract_Mk_from_logits(model):
    """
    For any model, simply interpret the logits grid as M_k[a,b].
    i.e., M_k[a,b] = logits_k(a,b).
    This is exact for discrete domain, though not weight-analytic.
    """
    logits = full_logits_grid(model)  # [a,b,k]
    return logits  # same shape [MOD,MOD,MOD]


In [ ]:
Mk_add_bilin_w  = extract_Mk_bilinear_from_weights(bilinear_add)
Mk_add_bilin_act = extract_Mk_from_logits(bilinear_add)
Mk_add_relu_act  = extract_Mk_from_logits(relu_add)

Mk_mul_bilin_w  = extract_Mk_bilinear_from_weights(bilinear_mul)
Mk_mul_bilin_act = extract_Mk_from_logits(bilinear_mul)
Mk_mul_relu_act  = extract_Mk_from_logits(relu_mul)

In [ ]:
# ============================================================
# 9. Fourier analysis (addition) – entropy per class
# ============================================================

def fourier_entropy(Mk):
    """
    Mk: numpy array [MOD, MOD, MOD]  (a,b,k)
    Returns: entropies H_k for each k.
    """
    N = Mk.shape[0]
    # DFT matrix F[u,a] = exp(-2πi * u*a / N) / sqrt(N)
    n = np.arange(N)
    omega = np.exp(-2j * np.pi / N)
    F = omega ** np.outer(n, n) / np.sqrt(N)  # [N,N], complex

    H = []
    for k in range(N):
        M = Mk[:, :, k]
        M_hat = F @ M @ F.conj().T    # [N,N] complex
        power = np.abs(M_hat) ** 2
        power = power / power.sum()
        p = power.flatten()
        p = p[p > 0]                  # avoid log(0)
        H_k = -np.sum(p * np.log(p))
        H.append(H_k)
    return np.array(H)

# Compute entropies for addition
H_add_bilin = fourier_entropy(Mk_add_bilin_w)
H_add_relu  = fourier_entropy(Mk_add_relu_act)

print("Mean entropy (add, bilinear):", H_add_bilin.mean())
print("Mean entropy (add, relu):    ", H_add_relu.mean())


In [ ]:
# ============================================================
# 10. Plot entropy histograms & example Fourier heatmaps
# ============================================================

# Histogram comparison
plt.figure(figsize=(6,4))
plt.hist(H_add_bilin, bins=20, alpha=0.5, label="Bilinear")
plt.hist(H_add_relu,  bins=20, alpha=0.5, label="ReLU")
plt.xlabel("Spectral entropy H_k")
plt.ylabel("Count")
plt.title("Fourier entropy per class (addition)")
plt.legend()
plt.show()

# Example Fourier heatmaps for a chosen class, say k=10
k_example = 10
N = MOD
n = np.arange(N)
omega = np.exp(-2j * np.pi / N)
F = omega ** np.outer(n, n) / np.sqrt(N)

def fourier_heat(Mk, k):
    M = Mk[:, :, k]
    M_hat = F @ M @ F.conj().T
    return np.abs(M_hat)

plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.imshow(fourier_heat(Mk_add_bilin_w, k_example))
plt.title(f"Bilinear: |F M_{k_example} F*|")
plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(fourier_heat(Mk_add_relu_act, k_example))
plt.title(f"ReLU: |F M_{k_example} F*|")
plt.colorbar()

plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 11. SVD / effective rank analysis (multiplication)
# ============================================================

def svd_effective_rank(Mk, thresh=0.9):
    """
    Mk: [MOD,MOD,MOD]
    Returns: r_k effective rank per class.
    """
    N = Mk.shape[0]
    ranks = []
    for k in range(N):
        M = Mk[:, :, k]
        # SVD
        u, s, vT = np.linalg.svd(M, full_matrices=False)
        s2 = s ** 2
        total = s2.sum()
        cum = np.cumsum(s2)
        r = np.searchsorted(cum, thresh * total) + 1
        ranks.append(r)
    return np.array(ranks)

ranks_mul_bilin = svd_effective_rank(Mk_mul_bilin_w)
ranks_mul_relu  = svd_effective_rank(Mk_mul_relu_act)

print("Mean effective rank (mul, bilinear):", ranks_mul_bilin.mean())
print("Mean effective rank (mul, relu):    ", ranks_mul_relu.mean())


In [ ]:
# ============================================================
# 12. Plot rank histograms
# ============================================================

plt.figure(figsize=(6,4))
plt.hist(ranks_mul_bilin, bins=20, alpha=0.5, label="Bilinear")
plt.hist(ranks_mul_relu,  bins=20, alpha=0.5, label="ReLU")
plt.xlabel("Effective rank r_k (90% energy)")
plt.ylabel("Count")
plt.title("Effective rank per class (multiplication)")
plt.legend()
plt.show()


In [ ]:
# ============================================================
# 13. Ground-truth operator for addition and its entropy
# ============================================================

def true_Mk_add():
    """
    Construct the exact modular addition operator as Mk[a,b,k],
    with Mk[a,b,k] = 1 if k == (a+b mod MOD) else 0.
    """
    Mk = np.zeros((MOD, MOD, MOD), dtype=np.float64)
    for a in range(MOD):
        for b in range(MOD):
            k = (a + b) % MOD
            Mk[a, b, k] = 1.0
    return Mk

Mk_true_add = true_Mk_add()
H_true_add = fourier_entropy(Mk_true_add)

print("Mean entropy (true addition):", H_true_add.mean())
print("Std  entropy (true addition):", H_true_add.std())
print("Mean entropy (add, bilinear):", H_add_bilin.mean())
print("Mean entropy (add, relu):    ", H_add_relu.mean())


In [ ]:
# ============================================================
# 14. Entropy histogram: true vs bilinear vs ReLU (addition)
# ============================================================

plt.figure(figsize=(7,4))
plt.hist(H_true_add,  bins=20, alpha=0.5, label="True operator")
plt.hist(H_add_bilin, bins=20, alpha=0.5, label="Bilinear MLP")
plt.hist(H_add_relu,  bins=20, alpha=0.5, label="ReLU MLP")
plt.xlabel("Spectral entropy H_k")
plt.ylabel("Count")
plt.title("Fourier entropy per class (addition mod 97)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 15. Check Mk from weights vs Mk from logits (bilinear, add)
# ============================================================

Mk_add_bilin_w   = extract_Mk_bilinear_from_weights(bilinear_add)
Mk_add_bilin_act = extract_Mk_from_logits(bilinear_add)

diff_mean = np.abs(Mk_add_bilin_w - Mk_add_bilin_act).mean()
diff_max  = np.abs(Mk_add_bilin_w - Mk_add_bilin_act).max()
print("Mean |Mk_w - Mk_act| (add, bilinear):", diff_mean)
print("Max  |Mk_w - Mk_act| (add, bilinear):", diff_max)


In [ ]:
# ============================================================
# 16. Centered SVD + stricter effective rank for multiplication
# ============================================================

def svd_effective_rank_centered(Mk, thresh=0.9):
    """
    Center each M_k (remove row/col means) and compute effective rank.
    """
    N = Mk.shape[0]
    ranks = []
    for k in range(N):
        M = Mk[:, :, k].astype(np.float64)
        # double-centering
        M = M - M.mean(axis=0, keepdims=True)
        M = M - M.mean(axis=1, keepdims=True)
        M = M + M.mean()
        u, s, vT = np.linalg.svd(M, full_matrices=False)
        s2 = s**2
        total = s2.sum()
        cum = np.cumsum(s2)
        r = np.searchsorted(cum, thresh * total) + 1
        ranks.append(r)
    return np.array(ranks)

# Use logits-based Mk for both models for fairness
Mk_mul_bilin = Mk_mul_bilin_act   # from logits
Mk_mul_relu  = Mk_mul_relu_act    # from logits

r_90_b = svd_effective_rank_centered(Mk_mul_bilin, thresh=0.9)
r_99_b = svd_effective_rank_centered(Mk_mul_bilin, thresh=0.99)

r_90_r = svd_effective_rank_centered(Mk_mul_relu, thresh=0.9)
r_99_r = svd_effective_rank_centered(Mk_mul_relu, thresh=0.99)

print("Centered eff rank (mul, bilinear) 90%:", r_90_b.mean(), "+/-", r_90_b.std())
print("Centered eff rank (mul, relu)     90%:", r_90_r.mean(), "+/-", r_90_r.std())
print("Centered eff rank (mul, bilinear) 99%:", r_99_b.mean(), "+/-", r_99_b.std())
print("Centered eff rank (mul, relu)     99%:", r_99_r.mean(), "+/-", r_99_r.std())


In [ ]:
# ============================================================
# 17. Rank histograms (centered) for multiplication
# ============================================================

plt.figure(figsize=(7,4))
plt.hist(r_99_b, bins=range(1, max(r_99_b.max(), r_99_r.max())+2), 
         alpha=0.5, label="Bilinear (99%)", align="left")
plt.hist(r_99_r, bins=range(1, max(r_99_b.max(), r_99_r.max())+2),
         alpha=0.5, label="ReLU (99%)", align="left")
plt.xlabel("Effective rank r_k (99% energy, centered)")
plt.ylabel("Count")
plt.title("Effective rank per class (multiplication)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 18. Singular value decay plots for example classes (mul)
# ============================================================

def plot_sv_spectrum(Mk_bilin, Mk_relu, k_list=[0, 10, 20]):
    for k in k_list:
        M_b = Mk_bilin[:, :, k].astype(np.float64)
        M_r = Mk_relu[:, :, k].astype(np.float64)

        # (optionally) center as above
        for label, M in [("Bilinear", M_b), ("ReLU", M_r)]:
            M_c = M - M.mean(axis=0, keepdims=True)
            M_c = M_c - M_c.mean(axis=1, keepdims=True)
            M_c = M_c + M_c.mean()

            _, s, _ = np.linalg.svd(M_c, full_matrices=False)
            s_norm = s / s[0]

            plt.plot(s_norm, marker='o', label=f"{label}, k={k}")

        plt.yscale("log")
        plt.xlabel("Singular value index")
        plt.ylabel("σ_i / σ_0 (log scale)")
        plt.title(f"Singular value decay (multiplication, class k={k})")
        plt.legend()
        plt.tight_layout()
        plt.show()

plot_sv_spectrum(Mk_mul_bilin, Mk_mul_relu, k_list=[0, 10, 20])
